# 📊 Análisis de Datos de Estaciones de Carga Eléctrica - Oasis

## Resumen Ejecutivo para Presentación

Este notebook presenta un análisis completo de los datos de transacciones en estaciones de carga eléctrica, con visualizaciones diseñadas para apoyar la toma de decisiones estratégicas y operativas.

---

**Fecha:** Noviembre 2025  
**Autor:** Análisis de Datos Oasis  
**Objetivo:** Proporcionar insights accionables para optimizar operaciones y maximizar rentabilidad

## 📋 Configuración Inicial

In [2]:
# Importar librerías necesarias
import pandas as pd
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Configurar rutas
current_dir = os.path.abspath('.')
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Agregar el directorio de gráficos al path
graficos_dir = os.path.join(current_dir, 'graficos')
if graficos_dir not in sys.path:
    sys.path.insert(0, graficos_dir)

# Importar módulos de gráficos
try:
    from graficos import grafico_01_barras_estaciones as g01
    from graficos import grafico_02_distribucion_usuarios as g02
    from graficos import grafico_03_segmentacion_rfm as g03
    from graficos import grafico_04_clv_segmentos as g04
    from graficos import grafico_05_ingresos_mensuales as g05
    from graficos import grafico_06_patrones_horarios as g06
    from graficos import grafico_07_heatmap_uso as g07
    from graficos import grafico_08_top_estaciones as g08
    from graficos import grafico_09_comparacion_t1_t2 as g09
    from graficos import grafico_10_energia_estaciones as g10
    from graficos import grafico_11_retencion_usuarios as g11
    from utils import cargar_datos
    print("✅ Módulos importados correctamente")
except Exception as e:
    print(f"⚠️ Error al importar módulos: {e}")
    print("Intentando cargar datos directamente...")
    from utils import cargar_datos

# Cargar datos
try:
    df = cargar_datos('data/df_oasis_clean.csv')
    # Normalizar columna de ingresos si no existe
    if 'ingresos_cop' not in df.columns:
        if 'amount_transaction_cop' in df.columns:
            df['ingresos_cop'] = df['amount_transaction_cop']
        elif 'amount_transaction' in df.columns:
            df['ingresos_cop'] = df['amount_transaction'] / 100
    
    # Normalizar columna de fechas
    if 'start_date_time' in df.columns:
        df['start_date_time'] = pd.to_datetime(df['start_date_time'], errors='coerce')
    
    print(f"✅ Datos cargados: {len(df):,} transacciones")
    print(f"📅 Período: {df['start_date_time'].min()} a {df['start_date_time'].max()}")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")

✅ Módulos importados correctamente
✓ Datos cargados: 8,739 registros
✅ Datos cargados: 8,739 transacciones
📅 Período: 2025-01-01 18:01:27 a 2025-10-01 01:41:21


---

# 1️⃣ Transacciones por Estación

## 🎯 ¿Para qué sirve este gráfico?

Identifica **cuáles estaciones tienen mayor actividad** al mostrar el número de transacciones realizadas en cada punto de carga.

## 💼 Valor Empresarial

- **Optimización de Recursos**: Permite identificar estaciones con alto tráfico que pueden necesitar más capacidad o mantenimiento preventivo
- **Planificación de Inversión**: Ayuda a decidir dónde instalar nuevas estaciones basándose en patrones de demanda existentes
- **Detección de Problemas**: Identifica estaciones con bajo uso que pueden tener problemas técnicos o de ubicación
- **Balance de Red**: Facilita la redistribución de recursos entre estaciones para optimizar la experiencia del usuario

## 📈 Visualización

In [3]:
fig1 = g01.crear_grafico(df)
fig1.show()

---

# 2️⃣ Distribución de Usuarios por Frecuencia

## 🎯 ¿Para qué sirve este gráfico?

Segmenta a los usuarios según su **frecuencia de uso**: usuarios de una sola transacción, uso ocasional (2-5), uso regular (6-10) y uso frecuente (11+).

## 💼 Valor Empresarial

- **Estrategias de Retención**: Identifica el porcentaje de usuarios "one-time" para crear programas de fidelización
- **Diseño de Membresías**: Permite crear planes diferenciados según patrones de uso (prepago, suscripciones, etc.)
- **Análisis de Churn**: Detecta usuarios en riesgo de abandono para implementar acciones correctivas
- **Proyección de Ingresos**: Ayuda a estimar ingresos recurrentes vs. esporádicos
- **Marketing Segmentado**: Facilita campañas personalizadas por nivel de uso

## 📈 Visualización

In [8]:
df_dist = g02.preparar_distribucion_usuarios(df)
fig2 = g02.crear_grafico(df)  # El gráfico espera el df original, no el preparado
fig2.show()

---

# 3️⃣ Segmentación RFM de Clientes

## 🎯 ¿Para qué sirve este gráfico?

Clasifica usuarios usando el modelo **RFM** (Recency, Frequency, Monetary):
- **VIP**: Usuarios recientes, frecuentes y de alto valor
- **Leales**: Usuarios consistentes con buen valor
- **Potenciales**: Usuarios con margen de crecimiento
- **En riesgo**: Usuarios que requieren atención inmediata

## 💼 Valor Empresarial

- **Priorización de Recursos**: Concentra esfuerzos en segmentos de mayor impacto
- **Personalización de Servicios**: Crea experiencias diferenciadas por segmento (VIP lounge, descuentos, prioridad)
- **Prevención de Churn**: Identifica usuarios "en riesgo" antes de perderlos
- **Upselling/Cross-selling**: Desarrolla estrategias para convertir "Potenciales" en "VIP"
- **ROI de Marketing**: Optimiza inversión publicitaria según el valor del segmento

## 📈 Visualización

In [13]:
df_rfm = g03.calcular_rfm(df)
fig3 = g03.crear_grafico_2d(df_rfm)  # Función correcta
fig3.show()

---

# 4️⃣ Valor del Cliente por Segmento (CLV)

## 🎯 ¿Para qué sirve este gráfico?

Muestra el **Customer Lifetime Value (CLV) promedio** por segmento RFM, indicando cuánto ingreso genera cada tipo de cliente.

## 💼 Valor Empresarial

- **Cálculo de ROI**: Determina cuánto invertir en adquisición y retención por segmento
- **Priorización de Inversión**: Justifica presupuestos mayores para segmentos de alto CLV
- **Estrategia de Precios**: Diseña planes premium para usuarios VIP que generan más valor
- **Proyecciones Financieras**: Estima ingresos futuros basándose en la composición de la base de clientes
- **Benchmarking**: Compara el valor generado por diferentes segmentos para identificar oportunidades

## 📈 Visualización

In [6]:
fig4 = g04.crear_grafico(df)
fig4.show()


===== CLV POR SEGMENTO (COP) =====
       segment  usuarios  ingreso_total  ingreso_promedio
3          VIP       367    105668048.0     287923.836512
1       Leales       388     33849274.0      87240.396907
2  Potenciales       795     20223679.0      25438.589937
0    En riesgo       562      6650259.0      11833.201068



---

# 5️⃣ Evolución de Ingresos Mensuales

## 🎯 ¿Para qué sirve este gráfico?

Visualiza la **tendencia de ingresos mes a mes** para identificar patrones de crecimiento, estacionalidad y anomalías.

## 💼 Valor Empresarial

- **Planificación Financiera**: Proyecta ingresos futuros basándose en tendencias históricas
- **Detección de Estacionalidad**: Identifica meses pico y bajos para ajustar inventario y recursos
- **Evaluación de Campañas**: Mide el impacto de promociones y estrategias de marketing
- **Alertas Tempranas**: Detecta caídas inesperadas que requieren investigación inmediata
- **Comunicación con Stakeholders**: Demuestra crecimiento y salud financiera del negocio

## 📈 Visualización

In [7]:
fig5 = g05.crear_grafico(df)
fig5.show()

---

# 6️⃣ Patrones de Uso Horario

## 🎯 ¿Para qué sirve este gráfico?

Identifica las **horas del día con mayor demanda** de servicios de carga.

## 💼 Valor Empresarial

- **Optimización de Tarifas**: Implementa precios dinámicos (surge pricing) en horas pico
- **Gestión de Personal**: Asigna recursos humanos según demanda real por horario
- **Planificación de Mantenimiento**: Programa mantenimientos en horas de baja demanda
- **Gestión Energética**: Negocia mejores tarifas eléctricas basándose en patrones de consumo
- **Campañas de Incentivos**: Ofrece descuentos en horas valle para balancear la demanda

## 📈 Visualización

In [10]:
df_temporal = g06.preparar_datos_temporales(df)
matriz = g06.crear_matriz_heatmap(df_temporal)
fig6 = g06.crear_grafico(df_temporal, matriz)
fig6.show()

---

# 7️⃣ Mapa de Calor: Uso por Estación y Tiempo

## 🎯 ¿Para qué sirve este gráfico?

Visualiza patrones de uso **combinando estación y tiempo** mediante un heatmap que muestra concentraciones de actividad.

## 💼 Valor Empresarial

- **Expansión Estratégica**: Identifica zonas geográficas y horarios con demanda insatisfecha
- **Redistribución de Capacidad**: Mueve recursos de estaciones subutilizadas a zonas de alta demanda
- **Partnerships Estratégicos**: Identifica ubicaciones para alianzas con comercios cercanos
- **Análisis Competitivo**: Compara performance entre estaciones para identificar mejores prácticas
- **Customer Experience**: Reduce tiempos de espera balanceando carga entre estaciones

## 📈 Visualización

In [19]:
# Crear heatmap de uso por estación y hora del día
df_heatmap = df.copy()
df_heatmap['hora'] = df_heatmap['start_date_time'].dt.hour
df_heatmap['dia_semana'] = df_heatmap['start_date_time'].dt.dayofweek

# Crear matriz de transacciones por estación y hora
top_estaciones = df['evse_uid'].value_counts().head(10).index.tolist()
df_heatmap_top = df_heatmap[df_heatmap['evse_uid'].isin(top_estaciones)]

matriz_estacion_hora = df_heatmap_top.pivot_table(
    index='evse_uid',
    columns='hora',
    values='id',
    aggfunc='count',
    fill_value=0
)

import plotly.graph_objects as go

fig7 = go.Figure(data=go.Heatmap(
    z=matriz_estacion_hora.values,
    x=[f'{h}:00' for h in matriz_estacion_hora.columns],
    y=matriz_estacion_hora.index,
    colorscale='YlOrRd',
    hovertemplate='Estación: %{y}<br>Hora: %{x}<br>Transacciones: %{z}<extra></extra>'
))

fig7.update_layout(
    title='Mapa de Calor: Uso de Estaciones por Hora del Día',
    xaxis_title='Hora del Día',
    yaxis_title='Estación',
    height=500,
    template='plotly_white'
)

fig7.show()

---

# 8️⃣ Top Estaciones por Rendimiento

## 🎯 ¿Para qué sirve este gráfico?

Rankea las **estaciones más exitosas** considerando múltiples métricas: ingresos, transacciones, usuarios únicos y energía dispensada.

## 💼 Valor Empresarial

- **Best Practices**: Identifica características de las estaciones exitosas para replicar
- **Benchmarking Interno**: Establece KPIs basados en el rendimiento de los mejores
- **Decisiones de Cierre**: Evalúa si estaciones de bajo rendimiento deben cerrarse o reubicarse
- **Inversión Priorizada**: Concentra mejoras en estaciones que ya demuestran alto potencial
- **Negociaciones Comerciales**: Usa datos de top performers para atraer inversores y partners

## 📈 Visualización

In [15]:
# Top estaciones por rendimiento
metricas = g08.calcular_metricas_estaciones(df)
fig8 = g08.figura_barras_top(metricas, top_n=15)
fig8.show()

---

# 9️⃣ Comparación: Conectores Tipo 1 vs Tipo 2

## 🎯 ¿Para qué sirve este gráfico?

Compara el rendimiento de **diferentes tipos de conectores** en términos de uso, ingresos, energía y preferencia de usuarios.

## 💼 Valor Empresarial

- **Decisiones de Inventario**: Determina qué tipo de cargadores comprar para nuevas instalaciones
- **Estandarización**: Evalúa si conviene estandarizar en un solo tipo de conector
- **Pricing Diferenciado**: Ajusta precios según demanda y características de cada tipo
- **Predicción de Demanda**: Anticipa necesidades basándose en tendencias de adopción de vehículos
- **Ventaja Competitiva**: Ofrece el mix correcto de conectores para atraer más clientes

## 📈 Visualización

In [16]:
# Comparación Tipo 1 vs Tipo 2
# Verificar si hay datos de conectores
if 'connector_id' in df.columns:
    # Preparar datos por tipo
    df_tipos = df.copy()
    df_tipos['tipo_conector'] = df_tipos['connector_id'].apply(lambda x: f'Tipo {x}' if pd.notna(x) else 'Sin Tipo')
    
    # Calcular métricas por tipo
    metricas_tipos = df_tipos.groupby('tipo_conector').agg({
        'id': 'count',
        'user_id': 'nunique',
        'energy_kwh': 'sum',
        'ingresos_cop': 'sum'
    }).reset_index()
    metricas_tipos.columns = ['tipo_conector', 'transacciones', 'usuarios', 'energia_kwh', 'ingresos']
    
    # Crear gráfico de barras comparativo
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    fig9 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Transacciones', 'Usuarios Únicos', 'Energía (kWh)', 'Ingresos (COP)'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    colores = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    fig9.add_trace(go.Bar(x=metricas_tipos['tipo_conector'], y=metricas_tipos['transacciones'],
                          marker_color=colores, showlegend=False), row=1, col=1)
    fig9.add_trace(go.Bar(x=metricas_tipos['tipo_conector'], y=metricas_tipos['usuarios'],
                          marker_color=colores, showlegend=False), row=1, col=2)
    fig9.add_trace(go.Bar(x=metricas_tipos['tipo_conector'], y=metricas_tipos['energia_kwh'],
                          marker_color=colores, showlegend=False), row=2, col=1)
    fig9.add_trace(go.Bar(x=metricas_tipos['tipo_conector'], y=metricas_tipos['ingresos'],
                          marker_color=colores, showlegend=False), row=2, col=2)
    
    fig9.update_layout(height=600, title_text="Comparación de Conectores por Tipo", showlegend=False)
    fig9.show()
else:
    print("⚠️ No hay datos de tipos de conectores disponibles")

---

# 🔟 Eficiencia Energética por Estación

## 🎯 ¿Para qué sirve este gráfico?

Analiza la **eficiencia en el uso de energía** por estación, mostrando kWh dispensados, eficiencia por transacción y relación costo-energía.

## 💼 Valor Empresarial

- **Reducción de Costos**: Identifica estaciones con pérdidas energéticas para mantenimiento correctivo
- **Pricing Inteligente**: Ajusta tarifas basándose en costos energéticos reales por estación
- **Sostenibilidad**: Demuestra eficiencia energética para reportes ESG y certificaciones
- **Mantenimiento Predictivo**: Detecta degradación de equipos mediante análisis de eficiencia
- **Negociación con Proveedores**: Usa datos de consumo para negociar mejores tarifas eléctricas

## 📈 Visualización

In [17]:
# Eficiencia energética por estación
metricas_energia = df.groupby('evse_uid').agg({
    'energy_kwh': 'sum',
    'id': 'count',
    'ingresos_cop': 'sum'
}).reset_index()
metricas_energia.columns = ['evse_uid', 'energia_total_kwh', 'transacciones', 'ingresos_total']
metricas_energia['kwh_por_tx'] = metricas_energia['energia_total_kwh'] / metricas_energia['transacciones']
metricas_energia['cop_por_kwh'] = metricas_energia['ingresos_total'] / metricas_energia['energia_total_kwh']
metricas_energia = metricas_energia.sort_values('energia_total_kwh', ascending=False).head(15)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig10 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Energía Total por Estación (kWh)', 'Eficiencia (kWh por Transacción)'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

fig10.add_trace(go.Bar(
    x=metricas_energia['evse_uid'], 
    y=metricas_energia['energia_total_kwh'],
    marker_color='lightgreen',
    showlegend=False
), row=1, col=1)

fig10.add_trace(go.Bar(
    x=metricas_energia['evse_uid'], 
    y=metricas_energia['kwh_por_tx'],
    marker_color='orange',
    showlegend=False
), row=1, col=2)

fig10.update_xaxes(tickangle=45)
fig10.update_layout(height=500, title_text="Análisis de Eficiencia Energética")
fig10.show()

---

# 1️⃣1️⃣ Retención de Usuarios (Análisis de Cohortes)

## 🎯 ¿Para qué sirve este gráfico?

Mide la **retención de usuarios a lo largo del tiempo** mediante análisis de cohortes mensuales, mostrando qué porcentaje de usuarios regresa después de su primera transacción.

## 💼 Valor Empresarial

- **Medición de Lealtad**: Cuantifica la efectividad de estrategias de retención
- **Cálculo de LTV**: Mejora proyecciones de valor de vida del cliente basándose en retención real
- **Identificación de Problemas**: Detecta caídas de retención que indican problemas de servicio
- **ROI de Onboarding**: Evalúa si los esfuerzos de bienvenida convierten usuarios en clientes recurrentes
- **Benchmark Temporal**: Compara cohortes para identificar qué períodos tuvieron mejor onboarding
- **Estrategia de Crecimiento**: Balancea inversión entre adquisición y retención según datos reales

## 📈 Visualización

In [18]:
# Retención de usuarios (análisis de cohortes)
# Calcular primera y última transacción por usuario
user_activity = df.groupby('user_id').agg({
    'start_date_time': ['min', 'max', 'count']
}).reset_index()
user_activity.columns = ['user_id', 'primera_tx', 'ultima_tx', 'total_tx']
user_activity['cohorte'] = user_activity['primera_tx'].dt.to_period('M')

# Calcular retención
cohortes = user_activity.groupby('cohorte').size().reset_index(name='usuarios_nuevos')
usuarios_repetidores = user_activity[user_activity['total_tx'] > 1].groupby('cohorte').size().reset_index(name='usuarios_repetidores')
cohortes = cohortes.merge(usuarios_repetidores, on='cohorte', how='left').fillna(0)
cohortes['tasa_retencion'] = (cohortes['usuarios_repetidores'] / cohortes['usuarios_nuevos'] * 100).round(1)
cohortes['cohorte_str'] = cohortes['cohorte'].astype(str)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig11 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Usuarios Nuevos por Cohorte', 'Tasa de Retención (%)'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

fig11.add_trace(go.Bar(
    x=cohortes['cohorte_str'], 
    y=cohortes['usuarios_nuevos'],
    marker_color='skyblue',
    showlegend=False,
    text=cohortes['usuarios_nuevos'],
    textposition='outside'
), row=1, col=1)

fig11.add_trace(go.Bar(
    x=cohortes['cohorte_str'], 
    y=cohortes['tasa_retencion'],
    marker_color='coral',
    showlegend=False,
    text=cohortes['tasa_retencion'].apply(lambda x: f'{x}%'),
    textposition='outside'
), row=1, col=2)

fig11.update_xaxes(tickangle=45)
fig11.update_layout(height=500, title_text="Análisis de Retención de Usuarios por Cohortes")
fig11.show()

print(f"\n📊 Resumen de Retención:")
print(f"   • Tasa de retención promedio: {cohortes['tasa_retencion'].mean():.1f}%")
print(f"   • Total usuarios nuevos: {cohortes['usuarios_nuevos'].sum():.0f}")
print(f"   • Total usuarios que repiten: {cohortes['usuarios_repetidores'].sum():.0f}")


📊 Resumen de Retención:
   • Tasa de retención promedio: 56.0%
   • Total usuarios nuevos: 2112
   • Total usuarios que repiten: 1144


---

# 🎯 Conclusiones y Recomendaciones

## Hallazgos Clave

1. **Concentración de Demanda**: Las estaciones top generan una porción desproporcionada de ingresos
2. **Oportunidad de Retención**: Alto porcentaje de usuarios one-time representa oportunidad de mejora
3. **Patrones Temporales**: Existe clara concentración de uso en horarios específicos
4. **Segmentación Clara**: Los usuarios VIP y Leales generan significativamente más valor

## Acciones Recomendadas

### Corto Plazo (0-3 meses)
- Implementar programa de fidelización para reducir usuarios one-time
- Ajustar tarifas en horas pico para optimizar ingresos
- Realizar mantenimiento preventivo en top estaciones

### Mediano Plazo (3-6 meses)
- Expandir capacidad en estaciones de alta demanda
- Desarrollar estrategias específicas por segmento RFM
- Implementar precios dinámicos basados en demanda

### Largo Plazo (6-12 meses)
- Planificar nuevas ubicaciones basadas en análisis de demanda
- Estandarizar tipos de conectores según preferencias del mercado
- Desarrollar modelos predictivos de demanda y churn

---

## 📞 Próximos Pasos

Este análisis proporciona la base para decisiones data-driven. Se recomienda:
- Actualizar dashboards mensualmente
- Establecer KPIs basados en estos gráficos
- Implementar sistema de alertas para métricas críticas
- Realizar A/B testing de estrategias propuestas

---

**¡Gracias por su atención!**